In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = "{:,.2f}".format

from experiments import *

import sys

sys.path.insert(1, '../fuzzylearn/')

from fuzzylearn import *
from fuzzylearn.fuzzifiers import LinearFuzzifier, CrispFuzzifier,ExponentialFuzzifier,QuantileLinearPiecewiseFuzzifier, QuantileConstantPiecewiseFuzzifier
from fuzzylearn.kernel import GaussianKernel, LinearKernel, HyperbolicKernel, PolynomialKernel, HomogeneousPolynomialKernel
from fuzzylearn import solve_optimization_gurobi

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, RobustScaler, PowerTransformer, Normalizer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.manifold import TSNE
from umap import UMAP

/home/manueldileo/anaconda3/lib/python3.7/site-packages/umap/spectral.py:4: NumbaDeprecationWarning: No direct replacement for 'numba.targets' available. Visit https://gitter.im/numba/numba-dev to request help. Thanks!
  import numba.targets


In [2]:
df_intero = pd.read_excel("dataset/database_investiti_13.xlsx")
df_intero = df_intero.set_index("VERBALE")
df_intero.head()

,DATA,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Neurocranio,Splancnocranio,Telencefalo,...,Art. coxo-femorale dx,Art. coxo-femorale sx,Rotula o Ginocchio dx,Rotula o Ginocchio sx,Caviglia dx,Caviglia sx,arto superiore destro,arto superiore sinistro,arto inferiore destro,arto inferiore sinistro
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,1999-10-29,0,81,84.00,1.75,27.43,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2
85829,2000-01-14,1,69,69.00,1.62,26.29,1,4,4,4,...,0,0,0,0,0,0,0,0,0,0
85977,2000-03-10,1,71,67.00,1.55,27.89,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
86220,2000-06-14,1,54,60.00,1.59,23.73,1,4,0,0,...,0,0,0,0,0,0,0,0,0,0
86247,2000-06-22,1,78,69.00,1.67,24.74,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
columns_13 = list(df_intero.columns)[1:7] + ['Volta cranica','Base cranica','Splancnoc.','Tratto cervicale',\
                                           'Tratto toracico','Tratto lombare','Totale coste','Sterno in toto',\
                                           'Bacino'] + list(df_intero.columns)[-4:]

In [4]:
df_13 = df_intero[columns_13]
df_13

,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Volta cranica,Base cranica,Splancnoc.,Tratto cervicale,Tratto toracico,Tratto lombare,Totale coste,Sterno in toto,Bacino,arto superiore destro,arto superiore sinistro,arto inferiore destro,arto inferiore sinistro
VERBALE,,,,,,,,,,,,,,,,,,,
85567,0,81,84.00,1.75,27.43,0,0,1,0,0,1,0,10,0,4,0,0,1,2
85829,1,69,69.00,1.62,26.29,1,6,6,3,0,0,0,168,2,0,0,0,0,0
85977,1,71,67.00,1.55,27.89,1,2,2,0,0,0,0,13,1,0,0,0,0,0
86220,1,54,60.00,1.59,23.73,1,1,4,0,0,0,0,32,1,0,0,0,0,0
86247,1,78,69.00,1.67,24.74,1,1,2,0,0,0,0,38,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101618,1,82,80.20,1.62,30.56,0,0,2,0,0,0,0,0,0,0,0,0,0,0
101288E,1,70,75.60,1.57,30.67,0,0,3,0,0,0,0,6,0,0,0,0,0,0
E-97586,0,45,45.00,1.77,14.36,1,0,0,0,0,0,0,10,1,6,0,0,0,0


In [5]:
col_anagrafica = list(df_13.columns)[0:5]
y = df_13['Mezzo'].values
mu0 = [1 if not label else 0 for label in y]
col_lesioni = list(df_13.columns)[6:]

# All

In [6]:
from experiments import *

In [5]:
filejson = 'json_result/fuzzifiers_df13.json'

In [6]:
import logging

# create logger
f_logger = logging.getLogger(__name__)
f_logger.setLevel(logging.INFO)

# create console handler and set level to debug
file = logging.FileHandler('esperimenti_df13.log')
file.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s -%(message)s')

# add formatter to ch
file.setFormatter(formatter)

# add ch to logger
f_logger.addHandler(file)

In [9]:
ordini = [.01,.1,1,10,100,1000]
pg_ordini= {
    'c' : ordini,
    'k' : [GaussianKernel(sigma) for sigma in ordini] + [LinearKernel()] + \
          [HyperbolicKernel(1,o) for o in ordini]
}

In [10]:
X = df_13[col_anagrafica+col_lesioni].values

In [11]:
l = 'df13_MSOrdine'

In [12]:
ft = incidenti_fuzzifier_table(df_13,[col_anagrafica+col_lesioni],fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=[l],\
                               file_json=filejson)
ft.to_json(filejson)

In [13]:
l = 'df13_MSOrdine_0'

In [14]:
ft = incidenti_fuzzifier_table(df_13,[col_anagrafica+col_lesioni],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=[l],\
                               file_json=filejson)
ft.to_json(filejson)

# Lesioni

In [15]:
l = 'df13lesioni_MSOrdine'

In [16]:
ft = incidenti_fuzzifier_table(df_13,[col_lesioni],fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=[l],\
                               file_json=filejson)
ft.to_json(filejson)

In [17]:
l = 'df13lesioni_MSOrdine_0'

In [18]:
ft = incidenti_fuzzifier_table(df_13,[col_lesioni],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=[l],\
                               file_json=filejson)
ft.to_json(filejson)

In [19]:
ft

,df13_MSOrdine,df13_MSOrdine_0,df13lesioni_MSOrdine,df13lesioni_MSOrdine_0
QuantileConstantPiecewiseFuzzifier,-0.42,-0.28,-0.41,-0.31
CrispFuzzifier,-0.48,-0.35,-0.55,-0.39
ExponentialFuzzifier,-0.30,-0.26,-0.30,-0.27
LinearFuzzifier,-0.30,-0.26,-0.30,-0.27
QuantileLinearPiecewiseFuzzifier,-0.40,-0.34,-0.41,-0.32


In [20]:
labels = ['df13lesioniPCA2','df13lesioniPCA5','df13lesioniPCA10','df13lesioniPCA13']

In [23]:
for c,l in zip([2,5,10,13],labels):
    ft = incidenti_fuzzifier_table(df_13,[col_lesioni],fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=[l],\
                               dim_reduction = PCA(n_components=c),file_json=filejson)
    ft.to_json(filejson)

In [10]:
l = 'df13lesioniPCA13_0'

In [11]:
ft = incidenti_fuzzifier_table(df_13,[col_lesioni],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=[l],\
                               dim_reduction = PCA(n_components=13),file_json=filejson)
ft.to_json(filejson)

# Totale Coste

In [54]:
l='df13_coste'

In [55]:
ft = incidenti_fuzzifier_table(df_13,[['Totale coste']],fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,labels=[l],scaling=None,file_json=filejson)
ft.to_json(filejson)

In [56]:
l='df13_coste_0'

In [57]:
pg_fine = {
    'c': [.1],
    'k': [GaussianKernel(sigma) for sigma in np.arange(.01,.1,.01)]
}

In [58]:
ft = incidenti_fuzzifier_table(df_13,[['Totale coste']],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,labels=[l],scaling=None,file_json=filejson)
ft.to_json(filejson)

In [59]:
ft

,df13_MSOrdine,df13_MSOrdine_0,df13lesioni_MSOrdine,df13lesioni_MSOrdine_0,df13lesioniPCA2,df13lesioniPCA5,df13lesioniPCA10,df13lesioniPCA13,df13lesioniPCA13_0,df13_coste,df13_coste_0
QuantileConstantPiecewiseFuzzifier,-0.42,-0.28,-0.41,-0.31,-0.65,-0.55,-0.43,-0.41,-0.31,-0.40,-0.40
CrispFuzzifier,-0.48,-0.35,-0.55,-0.39,-0.56,-0.62,-0.55,-0.55,-0.39,-0.53,-0.41
ExponentialFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.31,-0.30,-0.30,-0.30,-0.27,-0.31,-0.25
LinearFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.33,-0.31,-0.31,-0.30,-0.27,-0.34,-0.29
QuantileLinearPiecewiseFuzzifier,-0.40,-0.34,-0.41,-0.32,-0.65,-0.55,-0.43,-0.40,-0.30,-0.44,-0.39


# PCA 1 Gruppi sensati

In [60]:
anagrafica = df_13[list(df_13.columns)[0:5]]
etichette = df_13['Mezzo']
lesioni = df_13[list(df_13.columns)[6:]]

In [61]:
col_zone = ['Testa','Schiena','Addome','Arti']

In [62]:
df_compresso = anagrafica.copy()
for i,l in zip(range(0,12,3),col_zone):
    if i==9: zona = lesioni[list(lesioni.columns)[i:]]
    else: zona = lesioni[list(lesioni.columns)[i:(i+3)]]
    component = PCA(n_components=1).fit_transform(zona.values)
    component_to_values = [v[0] for v in component]
    serie = pd.Series(component_to_values,index=zona.index)
    df_compresso[l] = serie
    

In [63]:
df_compresso

,SESSO,ANNI,PESO,ALTEZZA,BMI,Testa,Schiena,Addome,Arti
VERBALE,,,,,,,,,
85567,0,81,84.00,1.75,27.43,-1.92,0.38,-19.41,1.14
85829,1,69,69.00,1.62,26.29,6.42,-0.62,138.50,-0.76
85977,1,71,67.00,1.55,27.89,0.05,-0.62,-16.48,-0.76
86220,1,54,60.00,1.59,23.73,0.70,-0.62,2.52,-0.76
86247,1,78,69.00,1.67,24.74,-0.61,-0.62,8.52,-0.76
...,...,...,...,...,...,...,...,...,...
101618,1,82,80.20,1.62,30.56,-1.26,-0.62,-29.48,-0.76
101288E,1,70,75.60,1.57,30.67,-0.61,-0.62,-23.48,-0.76
E-97586,0,45,45.00,1.77,14.36,-2.57,-0.62,-19.37,-0.76


## Lesioni

In [65]:
lesioniPCA = list(df_compresso.columns)[5:]

In [66]:
l = 'df13PCA1Lesioni'

In [67]:
ft = incidenti_fuzzifier_table(df_compresso,[lesioniPCA],fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,labels=[l],scaling=StandardScaler(),\
                               file_json=filejson)
ft.to_json(filejson)

/home/manueldileo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:788: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.newaxis]) ** 2,
100%|██████████| 100/100 [00:29<00:00,  3.44it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:20<00:00,  4.83it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:21<00:00,  4.75it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:20<00:00,  4.89it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/

In [68]:
l = 'df13PCA1Lesioni_0'

In [69]:
ft = incidenti_fuzzifier_table(df_compresso,[lesioniPCA],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,labels=[l],scaling=StandardScaler(),\
                               file_json=filejson)
ft.to_json(filejson)

/home/manueldileo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:788: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.newaxis]) ** 2,
100%|██████████| 100/100 [00:28<00:00,  3.47it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:20<00:00,  4.82it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:20<00:00,  4.80it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:21<00:00,  4.71it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/

## All

In [70]:
l = 'df13PCA1'

In [71]:
ft = incidenti_fuzzifier_table(df_compresso,[list(df_compresso.columns)],fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,labels=[l],scaling=StandardScaler(),\
                               file_json=filejson)
ft.to_json(filejson)

/home/manueldileo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:788: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.newaxis]) ** 2,
100%|██████████| 100/100 [00:29<00:00,  3.34it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:20<00:00,  4.88it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:21<00:00,  4.64it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:21<00:00,  4.76it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/

In [72]:
l = 'df13PCA1_0'

In [73]:
ft = incidenti_fuzzifier_table(df_compresso,[list(df_compresso.columns)],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,labels=[l],scaling=StandardScaler(),\
                               file_json=filejson)
ft.to_json(filejson)

/home/manueldileo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:788: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.newaxis]) ** 2,
100%|██████████| 100/100 [00:30<00:00,  3.31it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:21<00:00,  4.76it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:21<00:00,  4.63it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:21<00:00,  4.76it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/

In [74]:
ft

,df13_MSOrdine,df13_MSOrdine_0,df13lesioni_MSOrdine,df13lesioni_MSOrdine_0,df13lesioniPCA2,df13lesioniPCA5,df13lesioniPCA10,df13lesioniPCA13,df13lesioniPCA13_0,df13_coste,df13_coste_0,df13PCA1Lesioni,df13PCA1Lesioni_0,df13PCA1,df13PCA1_0
QuantileConstantPiecewiseFuzzifier,-0.42,-0.28,-0.41,-0.31,-0.65,-0.55,-0.43,-0.41,-0.31,-0.40,-0.40,-0.53,-0.31,-0.43,-0.28
CrispFuzzifier,-0.48,-0.35,-0.55,-0.39,-0.56,-0.62,-0.55,-0.55,-0.39,-0.53,-0.41,-0.56,-0.37,-0.45,-0.37
ExponentialFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.31,-0.30,-0.30,-0.30,-0.27,-0.31,-0.25,-0.31,-0.25,-0.30,-0.27
LinearFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.33,-0.31,-0.31,-0.30,-0.27,-0.34,-0.29,-0.32,-0.26,-0.29,-0.27
QuantileLinearPiecewiseFuzzifier,-0.40,-0.34,-0.41,-0.32,-0.65,-0.55,-0.43,-0.40,-0.30,-0.44,-0.39,-0.53,-0.32,-0.42,-0.31


In [75]:
l = 'df13PCA1LesioniMSFine_0'

In [76]:
pgfine0 = {
    'c': [.1],
    'k': [HyperbolicKernel(1,o) for o in range(0,10)]
}

In [79]:
ft = incidenti_fuzzifier_table(df_compresso,[lesioniPCA],fuzzifiers_class,FuzzyInductor,mu0,\
                               pgfine0, 3,3,logger=f_logger,labels=[l],scaling=StandardScaler(),\
                               file_json=filejson)
ft.to_json(filejson)

In [80]:
ft

,df13_MSOrdine,df13_MSOrdine_0,df13lesioni_MSOrdine,df13lesioni_MSOrdine_0,df13lesioniPCA2,df13lesioniPCA5,df13lesioniPCA10,df13lesioniPCA13,df13lesioniPCA13_0,df13_coste,df13_coste_0,df13PCA1Lesioni,df13PCA1Lesioni_0,df13PCA1,df13PCA1_0,df13PCA1LesioniMSFine_0
QuantileConstantPiecewiseFuzzifier,-0.42,-0.28,-0.41,-0.31,-0.65,-0.55,-0.43,-0.41,-0.31,-0.40,-0.40,-0.53,-0.31,-0.43,-0.28,-0.31
CrispFuzzifier,-0.48,-0.35,-0.55,-0.39,-0.56,-0.62,-0.55,-0.55,-0.39,-0.53,-0.41,-0.56,-0.37,-0.45,-0.37,-0.37
ExponentialFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.31,-0.30,-0.30,-0.30,-0.27,-0.31,-0.25,-0.31,-0.25,-0.30,-0.27,-0.25
LinearFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.33,-0.31,-0.31,-0.30,-0.27,-0.34,-0.29,-0.32,-0.26,-0.29,-0.27,-0.26
QuantileLinearPiecewiseFuzzifier,-0.40,-0.34,-0.41,-0.32,-0.65,-0.55,-0.43,-0.40,-0.30,-0.44,-0.39,-0.53,-0.32,-0.42,-0.31,-0.32


In [7]:
ris13 = pd.read_json(filejson)
ris13

,df13_MSOrdine,df13_MSOrdine_0,df13lesioni_MSOrdine,df13lesioni_MSOrdine_0,df13lesioniPCA2,df13lesioniPCA5,df13lesioniPCA10,df13lesioniPCA13,df13lesioniPCA13_0,df13_coste,df13_coste_0,df13PCA1Lesioni,df13PCA1Lesioni_0,df13PCA1,df13PCA1_0,df13PCA1LesioniMSFine_0
QuantileConstantPiecewiseFuzzifier,-0.42,-0.28,-0.41,-0.31,-0.65,-0.55,-0.43,-0.41,-0.31,-0.40,-0.40,-0.53,-0.31,-0.43,-0.28,-0.31
CrispFuzzifier,-0.48,-0.35,-0.55,-0.39,-0.56,-0.62,-0.55,-0.55,-0.39,-0.53,-0.41,-0.56,-0.37,-0.45,-0.37,-0.37
ExponentialFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.31,-0.30,-0.30,-0.30,-0.27,-0.31,-0.25,-0.31,-0.25,-0.30,-0.27,-0.25
LinearFuzzifier,-0.30,-0.26,-0.30,-0.27,-0.33,-0.31,-0.31,-0.30,-0.27,-0.34,-0.29,-0.32,-0.26,-0.29,-0.27,-0.26
QuantileLinearPiecewiseFuzzifier,-0.40,-0.34,-0.41,-0.32,-0.65,-0.55,-0.43,-0.40,-0.30,-0.44,-0.39,-0.53,-0.32,-0.42,-0.31,-0.32


In [10]:
best13 = ['df13_MSOrdine_0','df13lesioni_MSOrdine_0','df13lesioniPCA13_0','df13_coste_0']
ris13best = ris13[best13]
ris13best

,df13_MSOrdine_0,df13lesioni_MSOrdine_0,df13lesioniPCA13_0,df13_coste_0
QuantileConstantPiecewiseFuzzifier,-0.28,-0.31,-0.31,-0.40
CrispFuzzifier,-0.35,-0.39,-0.39,-0.41
ExponentialFuzzifier,-0.26,-0.27,-0.27,-0.25
LinearFuzzifier,-0.26,-0.27,-0.27,-0.29
QuantileLinearPiecewiseFuzzifier,-0.34,-0.32,-0.30,-0.39
